In [17]:
import os

GOOGLE_APPLICATION_CREDENTIALS="./user-gcp-sa.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = GOOGLE_APPLICATION_CREDENTIALS

In [18]:
#!pip3 install -r requirements.txt

In [19]:
import kfp
client = kfp.Client(host='4709991555f6b0ae-dot-us-central1.notebooks.googleusercontent.com')

# AutoML Serverless ML (Taxi) on KFP

#TODO: set GCS location to experiment run
#TODO: set num passengers out of categorical
TODO: run on unsampled datset
TODO: set split column to use fingerprint
TODO: Difference between table and dataset?
TODO: try using a cleanup task with boolean in testing mode
TODO: does generatinge new stats do anything?
TODO: How to add name to condition
TODO: Existing dataset as '' vs None?
TODO: good documentation on how to trigger code after conditional loop if the conditional doesn't run, we working on more traditional code logic?
TODO: in general, make clear what steps have inputs / outputs when or when they're not executed? get confusing to translate between argo and general python, None, etc
TODO: AUtoML, wanted to make more modular, had to refactor becuase you need to supply target column path, not name, couldnt' do my shortcut where i skipped dataset creation, ideally you only know a priori dataset, and then column names, not full pahts?
(might be more an issue with the op), also seems easies with new tables API?
TODO: AutoML Tables, why is it in core when not supported?
TODO: operand, couldn't compare with None so had to go to emtpy string
AttributeError: 'NoneType' object has no attribute 'id' (happens when client not initiated or stale?)
==None but not is None is supported

Wanted to do some logic on dataset_create_id, but it has to basically be reference as input to another step (not condition) or it will only be an artifact, not a paramter?
also, when i provide a paramater as teh input for another step, the variable name gets replaced in the UI as the paramater id, not the input name?


Ultimatey understanding Argo helped a lot
exit handler variations: https://argoproj.github.io/docs/argo/examples/README.html#exit-handlers

https://github.com/sebinsua/k8s-argo-parallel-aggregate-workflow/blob/master/parallel-aggregate-workflow.yml
https://github.com/argoproj/argo/issues/934

how to programtically delete a model that is training?

need for really basic python params, like LIMIT or adding to a query

bigquery op needs to output the temporary table? so i can suck it into AUtoML tables?

this is ok though?
gs://{{inputs.parameters.gcs_bucket}}/{{inputs.parameters.gcs_temp_directory}}/bq_taxi_output-*.csv
output_gcs_path='gs://{}/{}/bq_taxi_output-*.csv'.format(gcs_bucket, gcs_temp_directory)

why do outputs not put out more in general? i.e. BQ one shows me all the files, as well as the fuzzy match?



In [20]:
# Configuration

PROJECT_ID = 'dhodun1'
COMPUTE_REGION = 'us-central1' # Currently us-central1 is only region
BUCKET = 'dhodun1-central1'

# Raw dataset, not cleaned
QUERY = '''
SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers
FROM `nyc-tlc.yellow.trips`
WHERE MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 100000) = 1
'''


## Create Op to change 'passengers' from categorical to numeric
Should be fixed with *1.0 in BQ Query, but related to this BQ CSV export bug: https://b.corp.google.com/issues/143356550

In [21]:
# Test on training model, can I delete it?
from google.cloud import automl_v1beta1
client = automl_v1beta1.TablesClient()

client.get_model_evaluation(model_evaluation_name)

# models = [s for s in client.list_models(project=PROJECT_ID)]
# for model in models:
#     # print(model.deployment_state)
#     print(model.dataset_id)


NameError: name 'model_evaluation_name' is not defined

In [22]:
from typing import NamedTuple
from kfp import dsl
from kfp import components

def set_automl_tables_column_type(
    dataset_path: str,
    column_display_name: str,
    type_code: str,    
):
    # Updates AutuML Column with new column type, does trigger a new column statistics job? how do we check?
    import sys
    import subprocess
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'google-cloud-automl==0.9.0', '--quiet', '--no-warn-script-location'], env={'PIP_DISABLE_PIP_VERSION_CHECK': '1'}, check=True)
    
    from google.cloud import automl_v1beta1
    client = automl_v1beta1.TablesClient()
    
    dataset = client.get_dataset(dataset_name=dataset_path)
    
    column_specs_response = client.list_column_specs(dataset=dataset)
    column_specs = list(column_specs_response)
    
    for column in column_specs:
        if column.display_name == column_display_name:
            # This kicks off a new statistics job... how to check to see if it's done? Took ~ 1 minute this time
            response = client.update_column_spec(column_spec_name=column.name, dataset=dataset, type_code=type_code)
            print('Updated column: "{}" to type code {}. Generating new statistics now...'.format(column_display_name, type_code))

set_automl_tables_column_type_op = components.func_to_container_op(set_automl_tables_column_type, base_image='python:3.7')

In [23]:
# AutoML Tables components
! git clone https://github.com/kubeflow/pipelines.git


fatal: destination path 'pipelines' already exists and is not an empty directory.


In [24]:
from kfp import components

component_store = components.ComponentStore(local_search_paths=['./pipelines/components'])

In [25]:
automl_create_dataset_for_tables_op = component_store.load_component('gcp/automl/create_dataset_for_tables')
automl_import_data_from_bigquery_op = component_store.load_component('gcp/automl/import_data_from_bigquery')
automl_import_data_from_gcs_op = component_store.load_component('gcp/automl/import_data_from_gcs')
automl_create_model_for_tables_op = component_store.load_component('gcp/automl/create_model_for_tables')
prediction_service_batch_predict_op = component_store.load_component('gcp/automl/prediction_service_batch_predict')
automl_split_dataset_table_column_names_op = component_store.load_component('gcp/automl/split_dataset_table_column_names')

bigquery_query_op = component_store.load_component('gcp/bigquery/query')
#automl_create_dataset_for_tables_op = comp.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/b3179d86b239a08bf4884b50dbf3a9151da96d66/components/gcp/automl/create_dataset_for_tables/component.yaml')

In [26]:
#automl_create_dataset_for_tables_op(PROJECT_ID, COMPUTE_REGION, 'taxi_data',)

In [27]:
set_automl_tables_column_type(dataset_path=dataset_path, column_display_name='passengers', type_code='FLOAT64')

NameError: name 'dataset_path' is not defined

In [28]:
class ObjectDict(dict):
  def __getattr__(self, name):
    if name in self:
      return self[name]
    else:
      raise AttributeError("No such attribute: " + name)

# Define the pipeline

import kfp
from kfp import dsl

def serverless_automl_taxi(
    gcp_project_id: str,
    gcp_region: str,
    query: str,
    gcs_bucket: str,
    gcs_temp_directory: str = 'ml-taxi/{}'.format(dsl.RUN_ID_PLACEHOLDER),
    temp_table_name: str = '',
    dataset_display_name: str = 'taxi_data',
    dataset_location: str = 'US',
    existing_dataset_path: str = '',
    existing_dataset_id: str = '',
    target_column_name: str = 'fare_amount',
    model_display_name: str = 'taxi_data_model',
    train_budget_milli_node_hours: 'Integer' = 1000,
):
    """Pipeline to train on Taxi data"""

    new_dataset = True

    output_gcs_path='gs://{}/{}/bq_taxi_output-*.csv'.format(gcs_bucket, gcs_temp_directory)

    if new_dataset:

        # Create dataset
        create_dataset_task = automl_create_dataset_for_tables_op(
            gcp_project_id=gcp_project_id,
            gcp_region=gcp_region,
            display_name=dataset_display_name,
        )
        '''
        # Query to clean dataset and dump to GCS
        bigquery_export_task = bigquery_query_op(
            query=query,
            project_id=gcp_project_id,
            output_gcs_path=output_gcs_path,
            dataset_location=dataset_location,
        )'''

        # Query to clean dataset and dump to GCS
        bigquery_export_task = bigquery_query_op(
            query=query,
            project_id=gcp_project_id,
            dataset_location=dataset_location,
            table_id=temp_table_name,
        )
        '''
        # Import data from GCS automl_import_data_from_gcs_op
        import_data_task = automl_import_data_from_gcs_op(
            dataset_path=create_dataset_task.outputs['dataset_path'],
            input_uris=[output_gcs_path],
        ).after(bigquery_export_task)
        '''

        # Import data from BQ automl_import_data_from_gcs_op
        import_data_task = automl_import_data_from_bigquery_op(
            dataset_path=create_dataset_task.outputs['dataset_path'],
            input_uri=temp_table_name,
        ).after(bigquery_export_task)

        # Change 'passengers' column from categorical to numerical
        # Should be fixed with *1.0 in BQ Query, but related to this BQ CSV export bug: https://b.corp.google.com/issues/143356550
        set_passengers_numeric_task = set_automl_tables_column_type_op(
            dataset_path=import_data_task.outputs['dataset_path'],
            column_display_name='passengers',
            type_code='FLOAT64',
        )
    else:
        # Create the objects so that the rest of the pipeline will run
        create_dataset_task = ObjectDict({
            'outputs': {
                'dataset_id': existing_dataset_id
            }
        })
        import_data_task = ObjectDict({
            'outputs': {
                'dataset_path': existing_dataset_path
            }
        })


    # Prepare column schemas
    split_column_specs_task = automl_split_dataset_table_column_names_op(
        dataset_path=import_data_task.outputs['dataset_path'],
        table_index=0,
        target_column_name=target_column_name,        
    )
    if new_dataset:
        split_column_specs_task.after(set_passengers_numeric_task)

    # Train a model
    create_model_task = automl_create_model_for_tables_op(
        gcp_project_id=gcp_project_id,
        gcp_region=gcp_region,
        display_name=model_display_name,
        dataset_id=create_dataset_task.outputs['dataset_id'],
        target_column_path=split_column_specs_task.outputs['target_column_path'],
        # input_feature_column_paths=None, # All non-target columns will be used if None is passed
        # input_feature_column_paths=split_column_specs_task.outputs['feature_column_paths'],
        optimization_objective='MINIMIZE_RMSE',
        train_budget_milli_node_hours=train_budget_milli_node_hours,
    )
    
    from kfp.gcp import use_gcp_secret
    kfp.dsl.get_pipeline_conf().add_op_transformer(use_gcp_secret('user-gcp-sa'))

In [29]:
kfp.compiler.Compiler().compile(serverless_automl_taxi, 'pipeline.zip')

In [30]:
from google.cloud import automl

#PROJECT_ID = 'dhodun1'
#COMPUTE_REGION = 'us-central1' # Currently us-central1 is only region

#location_path = automl.AutoMlClient().location_path(PROJECT_ID, COMPUTE_REGION)
client = kfp.Client(host='4709991555f6b0ae-dot-us-central1.notebooks.googleusercontent.com')
client.create_run_from_pipeline_func(
    serverless_automl_taxi,
    arguments=dict(
        gcp_project_id=PROJECT_ID,
        gcp_region=COMPUTE_REGION,
        query=QUERY,
        gcs_bucket=BUCKET,
        dataset_display_name='taxi_data',
        existing_dataset_path='projects/978546835329/locations/us-central1/datasets/TBL9144959265608302592',
        existing_dataset_id='TBL9144959265608302592',
        temp_table_name='bq://dhodun1.kfp_tmp_dataset.taxi_export'
    )
)

ApiException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'Content-Length': '1452', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Tue, 03 Dec 2019 01:03:54 GMT', 'X-Content-Type-Options': 'nosniff', 'X-Frame-Options': 'SAMEORIGIN', 'X-Xss-Protection': '0', 'Set-Cookie': 'S=cloud_datalab_tunnel=4enqArZc86dm7BYyGvVwypMUGv2LTIZA; Path=/; Max-Age=3600'})
HTTP response body: 
<!DOCTYPE html>
<html lang=en>
  <meta charset=utf-8>
  <meta name=viewport content="initial-scale=1, minimum-scale=1, width=device-width">
  <title>Error 401 (Unauthorized)!!1</title>
  <style>
    *{margin:0;padding:0}html,code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{margin:7% auto 0;max-width:390px;min-height:180px;padding:30px 0 15px}* > body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 0 22px;overflow:hidden}ins{color:#777;text-decoration:none}a img{border:0}@media screen and (max

In [ ]:
!gcloud container clusters list

In [ ]:
!gcloud container clusters get-credentials kubeflow-marketplace-1 --zone us-central1-a

In [32]:
dataset_path = 'projects/978546835329/locations/us-central1/datasets/TBL5712864505831096320'
client = automl.AutoMlClient()
list_table_specs_response = client.list_table_specs(dataset_path)
list_table_specs_response
table_specs = [s for s in list_table_specs_response]
print('table_specs=')
print(table_specs)
table_spec_name = table_specs[0].name

list_column_specs_response = client.list_column_specs(table_spec_name)
column_specs = [s for s in list_column_specs_response]
#client.get_column_spec('passengers')
for column in column_specs:
    if column.display_name == 'passengers':
        passenger_column = column.name

print('column_specs=')
#print(column_specs)

column = client.get_column_spec(passenger_column)
print(column)

#client = automl_v1beta1.TablesClient()

client.update_column_spec(column_spec_name=passenger_column, type_code='NUMERIC')

AttributeError: 'AutoMlClient' object has no attribute 'list_table_specs'

In [ ]:
column.name

In [ ]:
table_specs[]

In [ ]:
kfp.run_pipeline_func_on_cluster(
    serverless_ml_taxi_pipeline,
    arguments = dict(
        gcp_project_id=PROJECT_ID,
        gcp_region=COMPUTE_REGION,
        display_name='taxi_data'
    )
)